In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
PATH = "/content/drive/MyDrive/Progetto Manutenzione"

Mounted at /content/drive


In [ ]:
# Caricare e concatenare i file CSV (Case001.csv-Case177.csv)
data_frames = []
for i in range(106, 154):
    file_path = (PATH + f'/dataset/train/data/Case{i:03d}.csv')
    df = pd.read_csv(file_path)
    df['Case'] = i
    data_frames.append(df)

data = pd.concat(data_frames, ignore_index=True)

column_names = ['Case', 'Spacecraft', 'Condition', 'SV1', 'SV2', 'SV3', 'SV4', 'BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BV1']
labels = pd.read_csv(PATH + '/dataset/train/labels.csv', names=column_names)

# Sostituisco "Normal" con 0, "Anomaly" e "Fault" con 1
labels['Condition'] = labels['Condition'].map({'Normal': 0, 'Anomaly': 1, 'Fault': 2})

# Sostituisco "No"/"Yes" con 0/1 nelle colonne da "BP1" a "BV1"
for col in ['BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BV1']:
    labels[col] = labels[col].map({'No': 0, 'Yes': 1})

In [ ]:
# Creazione dati di train
merged_data = pd.merge(data, labels, on="Case", how="left")
train_data = merged_data[["Case", "P1", "P2", "P3", "P4", "P5", "P6", "P7", "SV1", "SV2", "SV3", "SV4"]]

In [ ]:
train_data

,Case,P1,P2,P3,P4,P5,P6,P7,SV1,SV2,SV3,SV4
0,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100
1,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100
2,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100
3,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100
4,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100
...,...,...,...,...,...,...,...,...,...,...,...,...
57643,153,2.018495,2.018436,2.016806,2.173387,2.032607,1.948342,2.132867,100,100,100,75
57644,153,2.038382,2.043599,2.037118,1.893917,2.074023,2.175365,2.121340,100,100,100,75
57645,153,2.047800,2.063726,2.071593,2.289380,2.106856,2.223095,2.192724,100,100,100,75
57646,153,2.052519,2.076722,2.075243,1.901218,2.104139,2.003143,2.165096,100,100,100,75


In [ ]:
# Funzione per trovare il nome della colonna con valore diverso da 100
def find_diff_column(row):
    for col in ['SV1', 'SV2', 'SV3', 'SV4']:
        if row[col] != 100:
            return row[col]
    return 'None'

# Applico la funzione a ogni riga per creare la nuova colonna "Posizione"
train_data['Apertura'] = train_data.apply(find_diff_column, axis=1)

# Stampo le prime 5 righe del nuovo dataframe
train_data

<ipython-input-6-1faae8b8afe4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Apertura'] = train_data.apply(find_diff_column, axis=1)


,Case,P1,P2,P3,P4,P5,P6,P7,SV1,SV2,SV3,SV4,Apertura
0,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100,0.0
1,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100,0.0
2,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100,0.0
3,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100,0.0
4,106,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0,100,100,100,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57643,153,2.018495,2.018436,2.016806,2.173387,2.032607,1.948342,2.132867,100,100,100,75,75.0
57644,153,2.038382,2.043599,2.037118,1.893917,2.074023,2.175365,2.121340,100,100,100,75,75.0
57645,153,2.047800,2.063726,2.071593,2.289380,2.106856,2.223095,2.192724,100,100,100,75,75.0
57646,153,2.052519,2.076722,2.075243,1.901218,2.104139,2.003143,2.165096,100,100,100,75,75.0


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57648 entries, 0 to 57647
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Case      57648 non-null  int64  
 1   P1        57648 non-null  float64
 2   P2        57648 non-null  float64
 3   P3        57648 non-null  float64
 4   P4        57648 non-null  float64
 5   P5        57648 non-null  float64
 6   P6        57648 non-null  float64
 7   P7        57648 non-null  float64
 8   SV1       57648 non-null  int64  
 9   SV2       57648 non-null  int64  
 10  SV3       57648 non-null  int64  
 11  SV4       57648 non-null  int64  
 12  Apertura  57648 non-null  float64
dtypes: float64(8), int64(5)
memory usage: 5.7 MB


In [ ]:
features = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

X = train_data[features]
y = train_data[["Apertura"]]

In [ ]:
X

,P1,P2,P3,P4,P5,P6,P7
0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
3,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
4,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
...,...,...,...,...,...,...,...
57643,2.018495,2.018436,2.016806,2.173387,2.032607,1.948342,2.132867
57644,2.038382,2.043599,2.037118,1.893917,2.074023,2.175365,2.121340
57645,2.047800,2.063726,2.071593,2.289380,2.106856,2.223095,2.192724
57646,2.052519,2.076722,2.075243,1.901218,2.104139,2.003143,2.165096


In [ ]:
y

,Apertura
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
57643,75.0
57644,75.0
57645,75.0
57646,75.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

models = [
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('K-Nearest Neighbors', KNeighborsClassifier())
]

# Addestramento e valutazione dei modelli
for model_name, model in models:

    print(f"Training {model_name}...")
    model.fit(X_train, y_train)

    print(f"Prediction on {model_name}...")
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name}: Accuracy = {accuracy:.2f}')

Training Decision Tree...
Prediction on Decision Tree...
Decision Tree: Accuracy = 0.75
Training Random Forest...


<ipython-input-19-7b987cc61c25>:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Prediction on Random Forest...
Random Forest: Accuracy = 0.85
Training K-Nearest Neighbors...
Prediction on K-Nearest Neighbors...


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


K-Nearest Neighbors: Accuracy = 0.54


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertire y_train in un array 1D se necessario (solitamente non è necessario per classificazione, ma per sicurezza)
y_train_1d = np.ravel(y_train)

model_name = "Random Forest"
model = RandomForestClassifier()

print(f"Training {model_name}...")
model.fit(X_train, y_train_1d)

print(f"Prediction on {model_name}...")
y_pred = model.predict(X_test)

# Calcolare l'accuratezza per valutare il modello di classificazione
accuracy = accuracy_score(y_test, y_pred)
print(f'{model_name}: Accuracy = {accuracy:.2f}')


Training Random Forest...
Prediction on Random Forest...
Random Forest: Accuracy = 0.85


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertire y_train in un array 1D se necessario
y_train_1d = np.ravel(y_train)

model_name2 = "Random Forest Regressor"
model2 = RandomForestRegressor()

print(f"Training {model_name2}...")
model2.fit(X_train, y_train_1d)

print(f"Prediction on {model_name2}...")
y_pred = model2.predict(X_test)

# Calcolare il Mean Squared Error per valutare il modello di regressione
mse = mean_squared_error(y_test, y_pred)
print(f'{model_name2}: Mean Squared Error = {mse:.2f}')


Training Random Forest Regressor...
Prediction on Random Forest Regressor...
Random Forest Regressor: Mean Squared Error = 174.37


In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import mean_squared_error, r2_score

regression_models = [
    ('Linear Regression', LinearRegression()),
    ('Decision Tree Regressor', DecisionTreeRegressor()),
    ('Random Forest Regressor', RandomForestRegressor()),
    ('AdaBoost Regressor', AdaBoostRegressor()),
    ('Gradient Boosting Regressor', GradientBoostingRegressor()),
    ('Support Vector Regressor', SVR()),
    ('K-Nearest Neighbors Regressor', KNeighborsRegressor()),
    ('Bayesian Ridge', BayesianRidge()),
    ('Lasso', Lasso()),
    ('Ridge', Ridge()),
    ('ElasticNet', ElasticNet()),
    ('Multi-layer Perceptron Regressor', MLPRegressor())
]

# Addestramento e valutazione dei modelli
for model_name, model in models:
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    print(f"Prediction on {model_name}...")
    y_pred = model.predict(X_test)

    # Calcolo delle metriche di regressione
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f'{model_name}:')
    print(f'  Mean Squared Error (MSE): {mse:.2f}')
    print(f'  Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(f'  R-squared (R²): {r2:.2f}')

In [ ]:
class_model = models[2][1]
regress_model = regression_models[2][1]

print(class_model, regress_model)

RandomForestClassifier() RandomForestRegressor()


In [ ]:
#Creazione dati da classificare

# Caricare e concatenare i file CSV (Case178.csv-Case223.csv)
data_frames = []
for i in range(178, 223):
    file_path = (PATH + f'/dataset/test/data/Case{i:03d}.csv')
    df = pd.read_csv(file_path)
    df['Case'] = i
    data_frames.append(df)

prediction_data = pd.concat(data_frames, ignore_index=True)
prediction_data

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
...,...,...,...,...,...,...,...,...,...
54040,1.196,2.999206,2.985047,2.982110,2.970652,3.004048,3.009258,3.006424,222
54041,1.197,2.977775,2.952302,2.945279,2.933262,2.965413,2.964095,2.962946,222
54042,1.198,2.959111,2.930013,2.915038,2.926461,2.932936,2.921135,2.922863,222
54043,1.199,2.948195,2.921907,2.911228,2.918398,2.910006,2.895828,2.893770,222


In [ ]:
to_include = [179, 181, 188, 190, 199, 202, 205, 211, 212, 214]

prediction_data = prediction_data[prediction_data['Case'].isin(to_include)]
prediction_data

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
1201,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
1202,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
1203,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
1204,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
1205,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,179
...,...,...,...,...,...,...,...,...,...
44432,1.196,2.017808,2.014994,2.011121,1.997814,2.039541,2.101527,2.048623,214
44433,1.197,2.039463,2.050208,2.046588,2.031294,2.076301,2.139220,2.086852,214
44434,1.198,2.052856,2.076973,2.088993,2.054335,2.097308,2.149932,2.138660,214
44435,1.199,2.056427,2.092498,2.094648,2.094886,2.100509,2.151997,2.140353,214


In [ ]:
#CLASSIFICAZIONE

# Preparazione dei dati di test
features = prediction_data[["P1","P2","P3","P4","P5","P6","P7"]]
case_col = prediction_data[["Case"]]

# Predizione con il modello addestrato
predictions = model.predict(features)
features['Prediction'] = predictions

# Unione dei DataFrame
df_unique = pd.concat([case_col, features], axis=1)

# Calcolo della moda per ogni 'Case'
prediction_result = df_unique.groupby('Case')['Prediction'].apply(lambda x: x.mode().iat[0]).reset_index()

<ipython-input-28-90300b495495>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Prediction'] = predictions


In [ ]:
prediction_result
#188 -> 5, 199 -> 98, 202 -> 44, 205 -> 94, 211 -> 95

,Case,Prediction
0,179,25.0
1,181,75.0
2,188,25.0
3,190,50.0
4,199,75.0
5,202,0.0
6,205,75.0
7,211,0.0
8,212,75.0
9,214,25.0


In [ ]:
#PREDIZIONE

# Preparazione dei dati di test
features = prediction_data[["P1","P2","P3","P4","P5","P6","P7"]]
case_col = prediction_data[["Case"]]

# Predizione con il modello addestrato
predictions = model2.predict(features)
features['Prediction'] = predictions

# Unione dei DataFrame
df_unique = pd.concat([case_col, features], axis=1)

# Calcolo della moda per ogni 'Case'
prediction_result = df_unique.groupby('Case')['Prediction'].apply(lambda x: x.mean()).reset_index()

<ipython-input-30-db9c3915094e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Prediction'] = predictions


In [ ]:
prediction_result

,Case,Prediction
0,179,35.834108
1,181,57.427786
2,188,33.601171
3,190,49.390118
4,199,54.252198
5,202,36.186977
6,205,39.173993
7,211,38.914707
8,212,40.194329
9,214,32.528592
